# Tecnicas de prompt
Este código muestra cómo usar las técnicas de prompt para generar texto condicionado a un prompt.

## 1.- Setup inicial

### 1.1- Instalar librerías

In [ ]:
#! pip install openai
#! pip install tenacity
#! pip install python-dotenv

### 1.2.- Cargar librerías

In [ ]:
import openai
import os
import numpy as np
from datetime import datetime
from dotenv import load_dotenv
from tenacity import retry, wait_random_exponential, stop_after_attempt

### 1.3.- Variables de entorno

In [ ]:
# Load secrets and config from .env file
load_dotenv()

# OpenAI API
openai.api_key = os.getenv("OPENAI_API_KEY")
embedding_model = os.getenv("OPENAI_EMBEDDING_MODEL")
print("OpenAI API key: {}".format(openai.api_key[:5] + '...' + openai.api_key[-5:]))

# Model endpoint names
gpt35_model = os.getenv("OPENAI_GPT35_MODEL")
gpt35_16k_model = os.getenv("OPENAI_GPT35_16K_MODEL")
gpt4_model = os.getenv("OPENAI_GPT4_MODEL")
print("GPT-3.5-Turbo model: {}".format(gpt35_model))
print("GPT-3.5-Turbo-16k model: {}".format(gpt35_16k_model))
print("GPT-4 model: {}".format(gpt4_model))

### 1.4.- Clase para logs

In [ ]:
class Logger:
    def __init__(self, log_file=None, log_dir='./logs'):
        if not os.path.exists(log_dir):
            os.makedirs(log_dir)
        
        if log_file is None:
            timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
            log_file = f"log_{timestamp}.log"
        
        self.log_file = os.path.join(log_dir, log_file)

    def _write_log(self, level, msg, verbose):
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log_entry = f"{timestamp} [{level}] {msg}\n"
        with open(self.log_file, "a") as f:
            f.write(log_entry)
        if verbose:
            print(log_entry.strip())

    def info(self, msg, verbose=False):
        self._write_log("INFO", msg, verbose)

    def warn(self, msg, verbose=False):
        self._write_log("WARN", msg, verbose)

    def error(self, msg, verbose=True):
        self._write_log("ERROR", msg, verbose)
        
logger = Logger()
logger.info("Logger initialized", verbose=True)

### 1.5.- Clase para generación de texto con modelos GPT

In [ ]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(3))
def generate_text(prompt, model=gpt35_model, messages=[], max_tokens=100, temperature=1.0, top_p=1.0, frequency_penalty=0.0, presence_penalty=0.0, stop=None):
    _messages = []
    _messages.extend(messages)
    _messages.append({"role": "user", "content": prompt})
    
    _log_message = "\n\n============================ PROMPT ============================\n"
    for message in _messages:
        _log_message += f"{message['role']}: {message['content']}\n"
    logger.info(_log_message)
        
    response = openai.ChatCompletion.create(
        model=model,
        messages=_messages,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty,
        stop=stop
    )
    _log_message = "\n\n============================ RESPONSE ============================\n"
    _log_message += f"{response}\n"
    logger.info(_log_message)
    return response["choices"][0]["message"]["content"]

## 2.- Tecnicas

### 2.1.- Zero Shot

In [ ]:
prompt = """Clasifica el sentimiento del <TEXTO>. Positivo, Neutral o Negativo.
<TEXTO>
\"Me encanta la comida de este restaurante\""""
print(generate_text(prompt))

prompt = """<TEXTO>
\"Me encanta la comida de este restaurante\"
Clasifica el sentimiento del <TEXTO>. Positivo, Neutral o Negativo.
"""
print(generate_text(prompt))

system_message = {"role": "system", "content": "Clasifica el sentimiento del <TEXTO>. Positivo, Neutral o Negativo."}
prompt = """<TEXTO>
\"Me encanta la comida de este restaurante\"
"""
print(generate_text(prompt, messages=[system_message]))

In [ ]:
movie_review = """I wouldn't precisely say that Oppenheimer is a Christopher Nolan movie for people who don't like Christopher Nolan movies. I have people in my life who don't like Christopher Nolan movies, and they're still pretty cool towards this one. Maybe the way to put it is that Oppenheimer is the Christopher Nolan movie for people who in principle want to root for Christopher Nolan, but find his work in the last decade or so to be a bit too strange and off-putting and petulantly idiosyncratic, and have been hoping he'd return to telling stories about characters who say words that you can hear more or less clearly in the sound mix.

I am in no way, shape, or form such a person, and I will bluntly declare that I found Oppenheimer to be the director's least interesting work in probably 20 years, all the way back to Insomnia, in 2002. Which isn't to say that it's his worst film in that period, or even that it's not good. It is objectively, undeniably good, in fact. It's an across-the-board victory for all the below-the-line craftspeople that make movies an audiovisual art form, and if I may recklessly make the prediction, barely halfway through 2023, that this turns out to be the nomination leader for the Academy Awards this year, I doubt very much that I'll be able to find real fault with a single one of those nominations. It's also almost entirely well-acted; "almost" is in reference to a certain Nolan veteran cameoing as a certain President of the United States in such an unhinged ham sandwich of a performance that part of me assumes that "menacing camp" was the intended goal of his single scene, despite there not being a molecule of camp elsewhere in the film. But take out Gary Oldman's jaw-dropping Foghorn Leghorn of a Harry Truman, and I don't have a single substantive criticism for anybody in the indescribably well-stocked cast - on paper, among the most incredibly overqualified assemblages of actors in any American film of the 21st Century.

So Oppenheimer is, like, good. The only problem with it, if "problem" is the right way to put it, is that it is incredibly straightforward, and it's very tough for me not to sense a distinct "wounded puppy" vibe here, as if Nolan was taken aback by the cool reception towards his last film, 2020's Tenet, and wanted to prove what a good boy he could be so that he'd get a pat on the head. Speaking as a real big Tenet fan - it's possibly fair to call it the Nolan film I enjoy watching the most - I was never going to be sympathetic to that pivot, and so here we are. Regardless of whether this is a good thing or a bad thing or a thing that makes it one of the best films of the year, as the emerging consensus has it, Oppenheimer is pretty much precisely the movie it looks like from outside. And the movie it looks like from outside is a three-hour biopic of theoretical physicist J. Robert Oppenheimer (Cillian Murphy), whose main claim to fame was leading the Manhattan Project during World War II, in which basically every physicist of note living in the United States and some of the ones living in Great Britain were trucked to a highly secure and preposterously remote encampment in Los Alamos, New Mexico, to design the world's first nuclear bomb. And I think this, at least, is undeniably true: whatever good things this film does in three hours, I am fairly confident it would have been able to do all of them in two-and-a-half hours, also. Maybe not in a mere two hours, though I might have been willing to take the risk.

This is, to be fair, still a Nolan script (he adapted it from the Oppenheimer biography American Prometheus, by Kai Bird & Martin J. Sherwin), so it has a bit of structural wriggliness that keeps it from being entirely a cradle-to-grave story of one man's life. Within minutes of starting, the film informs us that it will be running two plot threads more or less simultaneously: "Fission", the shot-in-color story of how Oppenheimer has to think about his whole life before he can testify before an Atomic Energy Commission hearing in 1954, and "Fusion", the shot-in-black-and-white story of how the Oppenheimer affair in '54 affected the 1958 Senate hearing to determine whether former AEC chair Lewis Strauss (Robert Downey, Jr.) would be appointed as Secretary of Commerce. In practice, this ends up not being terribly complicated or elaborate: for most of the film's first two hours, the "Fusion" scenes feel like they're showing up mostly just to make sure we don't forget about Strauss and this will all eventually be going someplace, and it's really only during the last third that Strauss and his hostile relationship to Oppenheimer really begins to take up an equal share of the story. Meanwhile, "Fission" moves along pretty much start to finish: we see Oppenheimer's precocious college years, then skip ahead to his brash, arrogant young professor years, and then get to his Los Alamos years, all told in order, intercut with scenes of the middle-aged scientist looking utterly miserable as he tries to understand why these calm-sounding politicians keep badgering him with questions that have nothing at all to do with physics and everything to do with the leftist and Communist causes he was sort of tangled up with prior to the war.

Straightforward I call it, but at least Oppenheimer keeps a couple different balls in the air throughout all of this. Less, I suspect, as a matter of cunning strategy and more because Nolan is interested in the things he's interested in, the movie itself ends up demonstrating the same conflict it suggests rattled around inside its protagonist's heart. On the one hand, it really does want to try and explore the moral and political questions raised by Oppenheimer's extremely ambivalent life, going from fellow traveler to Death, Destroyer of Worlds, and to see what happens when a man who is absolutely happiest when he's letting purely theoretical ideas dance about in his head is obliged to grapple with what happens when his theories become practiced in the real world of living humans, finding that his studious amorality basically rounds down to being a wishy-washy asshole. On the other hand, it is just deeply interested in the nuts-and-bolts process of what happened at Los Alamos, an unabashedly nerdy dive through the history of quantum physics in the '30s and '40s that is so excited to include absolutely every even slightly well-known physicist of the first half of the 20th Century as a character, or a cameo, or at least a reference in dialogue, and to have them played by people who are decidedly too famous and/or talented to show up for two scenes and three lines, but I get that when Christopher Nolan comes calling, it's probably hard to turn down the gig. Most of the obvious "start cutting here" moments that separate my theoretical 2.5-hour Oppenheimer from the actually existing 3-hour Oppenheimer are these sort of "look at Richard Feynman (Jack Quaid) playing bongos!" and "I bet you didn't know that there was a woman scientist at Los Alamos, and her name was Lilli Hornig (Olivia Thirlby)" and "Did you ever hear about the time that Oppenheimer almost killed Niels Bohr (Kenneth Branagh) with a poisoned apple?" type of moments, basically just present because Nolan is enormously fascinated with the grand fabric of How We Got To The Atomic Age and thinks it's marvelous that the international world of theoretical physics in the pre-WWII era was basically just a small town where everybody knew everybody else. And he's not wrong, though the dramatic value of a lot of this material is basically nil.

So that's basically two movies, and then Oppenheimer goes ahead and adds a third entire movie when the Lewis Strauss material moves from being a small, almost negligible counterpoint (Nolan has said the black-and-white Strauss-oriented material is "objective" while the color Oppenheimer-oriented material is "subjective", and this doesn't actually feel to me like what the film is doing, but it does track with how much of the Oppenheimer material is in extremely shallow focus whereas I think literally none of the Strauss material is) to being a significant new narrative arc about how cynicism in politics is almost a fundamental physical force in its own right. And it's also pretty interesting, and benefits enormously from Downey giving his best performance in God-knows-how-many-years as Strauss, all bland competence and authoritative presence dissolving into serpentine viciousness, but it's a lot for the movie to try to combine in one body, and I don't think it gets there. Mostly, I think I will recall Oppenheimer as the laundry list of its best scenes, much more than the framework by which it stitches all of those scenes together.

And the best scenes are, to be fair, pretty great, and for several different reasons depending on the scene. Plenty of it comes down to the cast: Murphy, obliged to carry nearly all of such a long movie, does incredibly good work in making that look pretty easy, while also having to carry off the gimmick of aging from 22 to 59 (thanks to some of the best old-age makeup I have seen) and also describing the arc from hungry youth to unfathomably smug academic to gutted-out husk of a man who cannot comprehend the scope of his own foolishness and moral culpability. Probably every single thing the film does right, it could survive doing wrong - but not Murphy. If he wasn't constantly making this dizzy constellation of impulses feel like part of one cohesive lifetime (and in truth, I don't think there's a single bum note in his performance, though he can't quite sell the awkward blocking of a post-coital scene where Nolan can't figure out a natural way to keep him from flashing his penis to the camera, so he's just kind of contorting his pelvis away from us), Oppenheimer would be a meaningless grind through physics history trivia. And so even though several of the people around him get moments that are probably flashier - I am especially fond of a wordless "fuck you" look given in the film's last moments by Emily Blunt, whose outstanding performance as Oppenheimer's wife Kitty is significantly backloaded - Murphy has to start that list of the film's strengths.

It's also a pretty gorgeously-made thing. Literally gorgeous - cinematographer Hoyte Van Hoytema isn't doing anything as flashy as in his previous collaborations with Nolan, but the way he lets the hot, beating sun of the New Mexico desert dry out the film's color palette is rather effective, and it's just plain cool that he got Kodak to produce the first ever black-and-white IMAX stock, even if the film's use of color seems more purposeful and nuanced than its use of greyscale. But also, there's a level of clean, unobtrusive confidence and competence in all of the filmmaking, with certain elements rising to the foreground only when the film needs to affect us in some distinct way. Mostly, it's just making sure that everything works across that big void of a running time, with editor Jennifer Lame keeping the rhythms within scenes moving steadily enough that it never drags - it's not not a three-hour movie, but there aren't any places where it slows down, at least. Sometimes, it marshals itself for a singularly great moment: for example, the Trinity test, the explosion of the world's first atom bomb, which uses the simple trick of having the sound and Ludwig Göransson's score rage up louder and louder until it's almost unpleasant, and then cut everything out, so the visual of the explosion takes place against a complete void of silence, and in rather unspectacular wide shots, so the whole thing feels hollow and unfulfilling and horrifying. Or a scene in which Oppenheimer tries to offer a pep talk after the bombing of Hiroshima, and Van Hoytema carves the depth of field down to almost nothing and then shoots handheld, so it feels like Oppenheimer is so rattled and disgusted that the world is disintegrating and shaking itself apart around him. And then the sound again does a great deal to create a feeling of horror, between the high-pitched squeal of Murphy's voice, and the jarring inclusion of nondiegetic screams on the soundtrack, over the clattering babble of the Los Alamos scientists.

In short, this is very clear, effective, disciplined prestige filmmaking, enough to make this one of the toniest pieces of Oscarbait that I expect to see this year - but it is Oscarbait. It's a biopic, dressier than most, and with more things it's trying to put across, but there's still none of the "rip it all apart and see what we can assemble from the pieces" feeling that I've admired in Nolan's recent work. It's a fine movie: extremely fine. But I'm not certain that it's very interesting, and three hours is a very long time to not be certain of that."""

In [ ]:
prompt = f"""Clasifica el sentimiento del siguiente texto como Positivo, Neutral o Negativo:
{movie_review}"""
print(generate_text(prompt))

prompt = f"""{movie_review}
Clasifica el sentimiento del texto de arriba solo como Positivo, Neutral o Negativo:
"""
print(generate_text(prompt))

prompt = f"""Clasifica el sentimiento del <TEXTO>. Positivo, Neutral o Negativo.
<TEXTO>
{movie_review}"""
print(generate_text(prompt))

prompt = f"""<TEXTO>
{movie_review}
Clasifica el sentimiento del <TEXTO>. Positivo, Neutral o Negativo.
"""
print(generate_text(prompt))

system_message = {"role": "system", "content": "Clasifica el sentimiento del <TEXTO>. Positivo, Neutral o Negativo."}
prompt = f"""<TEXTO>
{movie_review}
"""
print(generate_text(prompt, messages=[system_message]))

In [ ]:
print(generate_text(prompt, messages=[system_message], model=gpt4_model))

In [ ]:
system_message = {"role": "system", "content": "Clasifica el sentimiento del <TEXTO>. Positivo, Neutral o Negativo. Primero entrega un razonamiento y luego tu respuesta final"}
prompt = f"""<TEXTO>
{movie_review}"""
print(generate_text(prompt, messages=[system_message], model=gpt4_model))

### 2.2.- Few Shot

In [ ]:
prompt = """La comida de este restaurante es muy buena. // Positivo
El servicio es muy malo. // Negativo
Hola, como estas? // Neutral
La edicion del video es clara y concisa. // """
print(generate_text(prompt))

In [ ]:
prompt = f"""La comida de este restaurante es muy buena. // Positivo
El servicio es muy malo. // Negativo
Hola, como estas? // Neutral
{movie_review} // """
print(generate_text(prompt))

### 2.3.- Chain of Thought

In [ ]:
system_message = {"role": "system", "content": "Solo responde con la respuesta final."}
prompt = f"""P: Quien vivio mas tiempo, Benjamin Franklin o Albert Einstein?

R: """
print(generate_text(prompt, messages=[system_message]))

In [ ]:
system_message = {"role": "system", "content": "Solo responde con la respuesta final."}
prompt = f"""P: Los directores de 'Jaws' y 'Casino Royale' son del mismo pais?

R: No

P: Quien vivio mas tiempo, Benjamin Franklin o Albert Einstein?

R: """
print(generate_text(prompt, messages=[system_message]))

In [ ]:
prompt = f"""P: Los directores de 'Jaws' y 'Casino Royale' son del mismo pais?
Pregunta Intermedia: Quien es el director de Jaws?
Respuesta Intermedia: El director de Jaws es Steven Spielberg.
Pregunta Intermedia: De que pais es Steven Spielberg?
Respuesta Intermedia: Steven Spielberg es de Estados Unidos.
Pregunta Intermedia: Quien es el director de Casino Royale?
Respuesta Intermedia: El director de Casino Royale es Martin Campbell.
Pregunta Intermedia: De que pais es Martin Campbell?
Respuesta Intermedia: Martin Campbell es de Nueva Zelanda.
Respuesta Final: No

P: Quien vivio mas tiempo, Benjamin Franklin o Albert Einstein?
"""
print(generate_text(prompt))

In [ ]:
prompt = f"""P: Quien vivio mas tiempo, Benjamin Franklin o Albert Einstein?
R: Pensemos paso a paso."""
print(generate_text(prompt))

### 2.4.- Self Consistency

In [ ]:
CONSISTENCY_ITERATIONS = 5

answers = []
prompt = f"""P: Quien vivio mas tiempo, Benjamin Franklin o Albert Einstein?
R: """

for i in range(CONSISTENCY_ITERATIONS):
    answer = generate_text(prompt, messages=[system_message])
    print(str(i+1) + ": " + answer + "\n=======================================")
    answers.append(answer)

# Select the most consistent answer using an LLM
answers_str = "\n".join([f"<RESPUESTA {i+1}>\n{answer}\n</RESPUESTA {i+1}>" for i, answer in enumerate(answers)])
prompt = f"""Tomando solamente en cuenta las <RESPUESTAS> Selecciona la <RESPUESTA> mas consistente (la que mas se repite):
{answers_str}
La respuesta mas consistente es la <RESPUESTA """
final_answer = generate_text(prompt, stop=">")

print(f"\nLa respuesta mas consistente es la <RESPUESTA {final_answer}>\n")
print(answers[int(final_answer)-1])

In [ ]:
CONSISTENCY_ITERATIONS = 5

answers = []
prompt = f"""P: Susan drives from city A to city B. After two hours of driving she noticed that she covered 80 km and calculated that, if she continued driving at the same speed, she would end up been 15 minutes late. So she increased her speed by 10 km/hr and she arrived at city B 36 minutes earlier than she planned.
Find the distance between cities A and B. 
R: """


for i in range(CONSISTENCY_ITERATIONS):
    answer = generate_text(prompt, messages=[system_message])
    print(str(i+1) + ": " + answer + "\n=======================================")
    answers.append(answer)

answers_str = "\n".join([f"<RESPUESTA {i+1}>\n{answer}\n</RESPUESTA {i+1}>" for i, answer in enumerate(answers)])
prompt = f"""Tomando solamente en cuenta las <RESPUESTAS> Selecciona la <RESPUESTA> mas consistente (la que mas se repite):
{answers_str}
La respuesta mas consistente es la <RESPUESTA """
final_answer = generate_text(prompt, stop=">")

print(f"\nLa respuesta mas consistente es la <RESPUESTA {final_answer}>\n")
print(answers[int(final_answer)-1])

In [ ]:
CONSISTENCY_ITERATIONS = 5

answers = []
prompt = """P: Mary, Peter, and Lucy were picking chestnuts. Mary picked twice as much chestnuts than Peter. Lucy picked 2 kg more than Peter. Together the three of them picked 26 kg of chestnuts. How many kilograms did each of them pick? 
R: """


for i in range(CONSISTENCY_ITERATIONS):
    answer = generate_text(prompt, messages=[system_message])
    print(str(i+1) + ": " + answer + "\n=======================================")
    answers.append(answer)

answers_str = "\n".join([f"<RESPUESTA {i+1}>\n{answer}\n</RESPUESTA {i+1}>" for i, answer in enumerate(answers)])
prompt = f"""Tomando solamente en cuenta las <RESPUESTAS> Selecciona la <RESPUESTA> mas consistente (la que mas se repite):
{answers_str}
La respuesta mas consistente es la <RESPUESTA """
final_answer = generate_text(prompt, stop=">")

print(f"\nLa respuesta mas consistente es la <RESPUESTA {final_answer}>\n")
print(answers[int(final_answer)-1])